# Setup

In [3]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import helpers
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, SimpleRNN, LSTM, Dense, Input, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Nadam

In [4]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [5]:
target = 'tmed'
np.random.seed(42)

In [6]:
n_days = 7
batch_size = 1024
metrics = ['mean_absolute_error', 'root_mean_squared_error', 'r2_score']

In [6]:
scaler_y_train = joblib.load('../../backend/data/ml/scalers/scaler_y_train.joblib')

In [7]:
train, validation, test, full = helpers.load_datasets()

In [8]:
train_dates = train.index
val_dates = validation.index
test_dates = test.index
full_dates = full.index

In [9]:
X_train = train.drop(columns=[target])
y_train = train[target]

X_val = validation.drop(columns=[target])
y_val = validation[target]

X_test = test.drop(columns=[target])
y_test = test[target]

X_full = full.drop(columns=[target])
y_full = full[target]

In [ ]:
X_train_seq, y_train_seq, train_dates_seq = helpers.create_sequences(X_train, y_train, train_dates, n_days)

In [10]:
X_val_seq, y_val_seq, val_dates_seq = helpers.create_sequences(X_val, y_val, val_dates, n_days)

In [11]:
X_test_seq, y_test_seq, test_dates_seq = helpers.create_sequences(X_test, y_test, test_dates, n_days)

In [10]:
X_full_seq, y_full_seq, full_dates_seq = helpers.create_sequences(X_full, y_full, full_dates, n_days)

In [11]:
shape = (n_days, X_full_seq.shape[2])

In [14]:
val_data = (X_val_seq, y_val_seq)

In [199]:
scaler = MinMaxScaler(feature_range=(0, 1))

# GRU

In [12]:
def create_gru(shape):

    model = Sequential()

    model.add(Input(shape=shape))

    model.add(GRU(units=50, return_sequences=True))
    model.add(GRU(units=50))

    model.add(Dense(units=1))

    return model

In [137]:
train_model_gru = create_gru(shape)
train_model_gru.compile(optimizer=Nadam(learning_rate=0.006), loss='mean_squared_error', metrics=metrics)

In [138]:
K.clear_session()
tf.compat.v1.reset_default_graph()
gru_history = train_model_gru.fit(X_train_seq, y_train_seq, epochs=10, batch_size=batch_size, validation_data=val_data)

Epoch 1/10
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0069 - mean_absolute_error: 0.0590 - r2_score: 0.5600 - root_mean_squared_error: 0.0790 - val_loss: 0.0050 - val_mean_absolute_error: 0.0558 - val_r2_score: 0.6884 - val_root_mean_squared_error: 0.0705
Epoch 2/10
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 0.0047 - mean_absolute_error: 0.0544 - r2_score: 0.7003 - root_mean_squared_error: 0.0685 - val_loss: 0.0047 - val_mean_absolute_error: 0.0539 - val_r2_score: 0.7047 - val_root_mean_squared_error: 0.0686
Epoch 3/10
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0042 - mean_absolute_error: 0.0507 - r2_score: 0.7327 - root_mean_squared_error: 0.0646 - val_loss: 0.0045 - val_mean_absolute_error: 0.0521 - val_r2_score: 0.7176 - val_root_mean_squared_error: 0.0671
Epoch 4/10
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0038 - mean_absolute_error: 0.0476 - r2_score: 0.7580 - root_mean_squared_error: 0.0615 - val_loss: 0.0045 - val_mean_absolute_error: 

In [239]:
gru_history_df = pd.DataFrame(gru_history.history)
gru_history_df.to_csv('../../frontend/resources/models/history/csv/gru.csv', index=False)
gru_history_df

,loss,mean_absolute_error,r2_score,root_mean_squared_error,val_loss,val_mean_absolute_error,val_r2_score,val_root_mean_squared_error
0,0.005113,0.055953,0.673245,0.071505,0.004972,0.055819,0.688355,0.070515
1,0.004577,0.053588,0.707534,0.067651,0.004712,0.053904,0.704669,0.068644
2,0.004056,0.049735,0.740784,0.063688,0.004505,0.052066,0.717637,0.067120
3,0.003738,0.047243,0.761122,0.061140,0.004503,0.051872,0.717747,0.067107
4,0.003596,0.046108,0.770207,0.059965,0.004328,0.050491,0.728715,0.065790
5,0.003507,0.045399,0.775885,0.059221,0.004312,0.050323,0.729741,0.065666
6,0.003447,0.044921,0.779756,0.058707,0.004262,0.050029,0.732851,0.065287
7,0.003402,0.044559,0.782626,0.058323,0.004305,0.050426,0.730181,0.065612
8,0.003365,0.044269,0.784980,0.058005,0.004291,0.050191,0.731088,0.065502
9,0.003336,0.044049,0.786817,0.057758,0.004225,0.049815,0.735198,0.065000


In [218]:
gru_history_df_scaled = pd.DataFrame(scaler.fit_transform(gru_history_df), columns=gru_history_df.columns)
gru_history_df_scaled

,loss,mean_absolute_error,r2_score,root_mean_squared_error,val_loss,val_mean_absolute_error,val_r2_score,val_root_mean_squared_error
0,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
1,0.698136,0.801346,0.301919,0.719595,0.651735,0.680934,0.348266,0.660856
2,0.405251,0.477672,0.594684,0.431337,0.374905,0.374865,0.625096,0.384537
3,0.226261,0.268345,0.773751,0.245987,0.372543,0.342578,0.627459,0.382153
4,0.146200,0.173005,0.853753,0.160532,0.138407,0.112509,0.861593,0.143407
5,0.096265,0.113439,0.903745,0.106375,0.116500,0.084620,0.883499,0.120825
6,0.062191,0.073272,0.937831,0.069025,0.050113,0.035528,0.949887,0.052124
7,0.036854,0.042823,0.963102,0.041039,0.107102,0.101698,0.892897,0.111122
8,0.016094,0.018498,0.983823,0.017970,0.087746,0.062555,0.912253,0.091117
9,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000


In [139]:
gru_metrics = train_model_gru.evaluate(X_test_seq, y_test_seq)

21849/21849 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 0.0048 - mean_absolute_error: 0.0531 - r2_score: 0.6449 - root_mean_squared_error: 0.0694


In [140]:
gru_y_pred_seq = train_model_gru.predict(X_test_seq)

21849/21849 ━━━━━━━━━━━━━━━━━━━━ 18s 801us/step


In [238]:
gru_metrics_df = helpers.metrics_df('GRU', gru_metrics)
gru_metrics_df.to_csv('../../frontend/resources/models/metrics/gru.csv', index=False)
gru_metrics_df

,Model,MSE,MAE,RMSE,R2
0,GRU,0.004981,0.054277,0.070575,0.689429


In [142]:
gru_y_pred, gru_y_test, gru_difference = helpers.inverse_predictions(gru_y_pred_seq, y_test_seq, scaler_y_train)

In [237]:
gru_daily_stats = helpers.daily_stats(gru_y_pred, gru_y_test, gru_difference, test_dates_seq)
gru_daily_stats.to_csv('../../frontend/resources/models/daily_stats/csv/gru.csv', index=False)
gru_daily_stats.head()

,fecha,actual_mean,actual_min,actual_max,actual_std,predicted_mean,predicted_min,predicted_max,predicted_std
0,2023-01-01,12.538392,1.70042,22.19802,3.418742,11.741107,6.294677,19.328323,1.734991
1,2023-01-02,10.296912,-0.80051,21.30125,4.043459,10.267680,3.226587,20.115284,1.807020
2,2023-01-03,9.300819,-2.19858,21.80255,4.564409,9.641691,2.649850,19.100275,2.221143
3,2023-01-04,9.353709,-0.49973,21.40151,4.595705,9.553359,3.820453,19.182291,2.047590
4,2023-01-05,8.972093,-0.80051,21.40151,4.497757,9.325142,2.494161,19.379030,2.110607


In [ ]:
gru_daily_stats_fig = helpers.daily_stats_comparison('GRU', gru_daily_stats)
gru_daily_stats_fig.write_html('../../frontend/resources/models/daily_stats/figures/gru.html')
gru_daily_stats_fig.show()

In [ ]:
gru_difference_fig = helpers.difference_histogram('GRU', gru_difference)
gru_difference_fig.write_html('../../frontend/resources/models/differences/gru.html')
gru_difference_fig.show()

In [ ]:
gru_training_history_fig = helpers.training_history('GRU', gru_history.history)
gru_training_history_fig.write_html('../../frontend/resources/models/history/figures/gru.html')
gru_training_history_fig.show()

In [ ]:
gru_training_history_scaled_fig = helpers.training_history_scaled('GRU', gru_history_df_scaled, gru_history_df)
gru_training_history_scaled_fig.write_html('../../frontend/resources/models/history/figures/gru_scaled.html')
gru_training_history_scaled_fig.show()

In [13]:
final_model_gru = create_gru(shape)
final_model_gru.compile(optimizer=Nadam(learning_rate=0.006), loss='mean_squared_error', metrics=metrics)

I0000 00:00:1741575519.084598   11544 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5983 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [14]:
K.clear_session()
tf.compat.v1.reset_default_graph()
final_model_gru.fit(X_full_seq, y_full_seq, epochs=8, batch_size=batch_size)

Epoch 1/8


2025-03-10 03:58:42.011354: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
I0000 00:00:1741575522.102609   11908 cuda_dnn.cc:529] Loaded cuDNN version 90600


4476/4476 ━━━━━━━━━━━━━━━━━━━━ 23s 5ms/step - loss: 0.0064 - mean_absolute_error: 0.0585 - r2_score: 0.5836 - root_mean_squared_error: 0.0772 
Epoch 2/8
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.0046 - mean_absolute_error: 0.0539 - r2_score: 0.6957 - root_mean_squared_error: 0.0682
Epoch 3/8
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.0041 - mean_absolute_error: 0.0500 - r2_score: 0.7304 - root_mean_squared_error: 0.0641
Epoch 4/8
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.0038 - mean_absolute_error: 0.0477 - r2_score: 0.7500 - root_mean_squared_error: 0.0618
Epoch 5/8
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.0037 - mean_absolute_error: 0.0467 - r2_score: 0.7581 - root_mean_squared_error: 0.0608
Epoch 6/8
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.0036 - mean_absolute_error: 0.0461 - r2_score: 0.7627 - root_mean_squared_error: 0.0602
Epoch 7/8
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 0.0036 - mean_absolute_error: 0.045

In [15]:
final_model_gru.save('../../backend/data/ml/models/gru.keras')

# SimpleRNN

In [46]:
def create_simplernn(shape):
    
    model = Sequential()
    
    model.add(Input(shape=shape))

    
    model.add(SimpleRNN(units=50, return_sequences=True))
    model.add(SimpleRNN(units=50))
    
    model.add(Dense(units=1))
    
    return model

In [85]:
train_model_simplernn = create_simplernn(shape)
train_model_simplernn.compile(optimizer=Nadam(learning_rate=0.001), loss='mean_squared_error', metrics=metrics)

In [86]:
K.clear_session()
tf.compat.v1.reset_default_graph()
simplernn_history = train_model_simplernn.fit(X_train_seq, y_train_seq, epochs=5, batch_size=batch_size, validation_data=val_data)

Epoch 1/5
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0080 - mean_absolute_error: 0.0637 - r2_score: 0.4891 - root_mean_squared_error: 0.0848 - val_loss: 0.0054 - val_mean_absolute_error: 0.0583 - val_r2_score: 0.6635 - val_root_mean_squared_error: 0.0733
Epoch 2/5
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0049 - mean_absolute_error: 0.0557 - r2_score: 0.6858 - root_mean_squared_error: 0.0701 - val_loss: 0.0051 - val_mean_absolute_error: 0.0564 - val_r2_score: 0.6833 - val_root_mean_squared_error: 0.0711
Epoch 3/5
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0048 - mean_absolute_error: 0.0550 - r2_score: 0.6942 - root_mean_squared_error: 0.0692 - val_loss: 0.0052 - val_mean_absolute_error: 0.0567 - val_r2_score: 0.6766 - val_root_mean_squared_error: 0.0718
Epoch 4/5
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0047 - mean_absolute_error: 0.0544 - r2_score: 0.6993 - root_mean_squared_error: 0.0686 - val_loss: 0.0049 - val_mean_absolute_error: 0.0553 

In [240]:
simplernn_history_df = pd.DataFrame(simplernn_history.history)
simplernn_history_df.to_csv('../../frontend/resources/models/history/csv/simplernn.csv', index=False)
simplernn_history_df

,loss,mean_absolute_error,r2_score,root_mean_squared_error,val_loss,val_mean_absolute_error,val_r2_score,val_root_mean_squared_error
0,0.005538,0.058092,0.646109,0.074415,0.005369,0.058289,0.663474,0.073276
1,0.004878,0.055524,0.688260,0.069843,0.005053,0.056448,0.683290,0.071085
2,0.004766,0.054841,0.695407,0.069039,0.005160,0.056659,0.676578,0.071835
3,0.004688,0.054322,0.700411,0.068470,0.004925,0.055273,0.691299,0.070181
4,0.004617,0.053826,0.704915,0.067952,0.004885,0.055118,0.693856,0.069890


In [219]:
simplernn_history_df_scaled = pd.DataFrame(scaler.fit_transform(simplernn_history_df), columns=simplernn_history_df.columns)
simplernn_history_df_scaled

,loss,mean_absolute_error,r2_score,root_mean_squared_error,val_loss,val_mean_absolute_error,val_r2_score,val_root_mean_squared_error
0,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
1,0.283164,0.397983,0.716791,0.292561,0.347760,0.419497,0.652241,0.353163
2,0.161762,0.238015,0.838321,0.168111,0.568689,0.485887,0.431310,0.574471
3,0.076710,0.116355,0.923421,0.080053,0.084143,0.048806,0.915856,0.086003
4,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000


In [87]:
simplernn_metrics = train_model_simplernn.evaluate(X_test_seq, y_test_seq)

21849/21849 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - loss: 0.0053 - mean_absolute_error: 0.0569 - r2_score: 0.6094 - root_mean_squared_error: 0.0727


In [88]:
simplernn_y_pred_seq = train_model_simplernn.predict(X_test_seq)

21849/21849 ━━━━━━━━━━━━━━━━━━━━ 19s 858us/step


In [236]:
simplernn_metrics_df = helpers.metrics_df('SimpleRNN', simplernn_metrics)
simplernn_metrics_df.to_csv('../../frontend/resources/models/metrics/simplernn.csv', index=False)
simplernn_metrics_df

,Model,MSE,MAE,RMSE,R2
0,SimpleRNN,0.005284,0.05695,0.072688,0.670547


In [171]:
simplernn_y_pred, simplernn_y_test, simplernn_difference = helpers.inverse_predictions(simplernn_y_pred_seq, y_test_seq, scaler_y_train)

In [235]:
simplernn_daily_stats = helpers.daily_stats(simplernn_y_pred, simplernn_y_test, simplernn_difference, test_dates_seq)
simplernn_daily_stats.to_csv('../../frontend/resources/models/daily_stats/csv/simplernn.csv', index=False)
simplernn_daily_stats.head()

,fecha,actual_mean,actual_min,actual_max,actual_std,predicted_mean,predicted_min,predicted_max,predicted_std
0,2023-01-01,12.538392,1.70042,22.19802,3.418742,11.701744,7.919505,18.245699,1.075005
1,2023-01-02,10.296912,-0.80051,21.30125,4.043459,10.470489,7.451316,14.641874,0.966112
2,2023-01-03,9.300819,-2.19858,21.80255,4.564409,9.833611,6.246911,16.821663,1.089722
3,2023-01-04,9.353709,-0.49973,21.40151,4.595705,9.801921,4.791089,16.304659,1.131123
4,2023-01-05,8.972093,-0.80051,21.40151,4.497757,9.547573,6.088550,16.119761,1.122485


In [ ]:
simplernn_daily_stats_fig = helpers.daily_stats_comparison('SimpleRNN', simplernn_daily_stats)
simplernn_daily_stats_fig.write_html('../../frontend/resources/models/daily_stats/figures/simplernn.html')
simplernn_daily_stats_fig.show()

In [ ]:
simplernn_difference_fig = helpers.difference_histogram('SimpleRNN', simplernn_difference)
simplernn_difference_fig.write_html('../../frontend/resources/models/differences/simplernn.html')
simplernn_difference_fig.show()

In [ ]:
simplernn_training_history_fig = helpers.training_history('SimpleRNN', simplernn_history.history)
simplernn_training_history_fig.write_html('../../frontend/resources/models/history/figures/simplernn.html')
simplernn_training_history_fig.show()

In [ ]:
simplernn_training_history_scaled_fig = helpers.training_history_scaled('SimpleRNN', simplernn_history_df_scaled, simplernn_history_df)
simplernn_training_history_scaled_fig.write_html('../../frontend/resources/models/history/figures/simplernn_scaled.html')
simplernn_training_history_scaled_fig.show()

In [95]:
final_model_simplernn = create_simplernn(shape)
final_model_simplernn.compile(optimizer=Nadam(learning_rate=0.001), loss='mean_squared_error', metrics=metrics)

In [96]:
K.clear_session()
tf.compat.v1.reset_default_graph()
final_model_simplernn.fit(X_full_seq, y_full_seq, epochs=5, batch_size=batch_size)

Epoch 1/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 0.0160 - mean_absolute_error: 0.0703 - r2_score: -0.0514 - root_mean_squared_error: 0.1074
Epoch 2/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.0050 - mean_absolute_error: 0.0559 - r2_score: 0.6745 - root_mean_squared_error: 0.0705
Epoch 3/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.0048 - mean_absolute_error: 0.0549 - r2_score: 0.6853 - root_mean_squared_error: 0.0693
Epoch 4/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.0047 - mean_absolute_error: 0.0542 - r2_score: 0.6921 - root_mean_squared_error: 0.0685
Epoch 5/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.0046 - mean_absolute_error: 0.0537 - r2_score: 0.6977 - root_mean_squared_error: 0.0679


In [97]:
final_model_simplernn.save('../../backend/data/ml/models/simplernn.keras')

# LSTM

In [13]:
def create_lstm(shape):
    
    model = Sequential()
    
    model.add(Input(shape=shape))
    
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))
    
    model.add(Dense(units=1))
    
    return model

In [112]:
train_model_lstm = create_lstm(shape)
train_model_lstm.compile(optimizer=Nadam(learning_rate=0.01), loss='mean_squared_error', metrics=metrics)

In [113]:
K.clear_session()
tf.compat.v1.reset_default_graph()
lstm_history = train_model_lstm.fit(X_train_seq, y_train_seq, epochs=8, batch_size=batch_size, validation_data=val_data)

Epoch 1/8
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - loss: 0.0067 - mean_absolute_error: 0.0601 - r2_score: 0.5705 - root_mean_squared_error: 0.0789 - val_loss: 0.0052 - val_mean_absolute_error: 0.0571 - val_r2_score: 0.6754 - val_root_mean_squared_error: 0.0720
Epoch 2/8
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0048 - mean_absolute_error: 0.0550 - r2_score: 0.6939 - root_mean_squared_error: 0.0692 - val_loss: 0.0049 - val_mean_absolute_error: 0.0554 - val_r2_score: 0.6924 - val_root_mean_squared_error: 0.0701
Epoch 3/8
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0044 - mean_absolute_error: 0.0526 - r2_score: 0.7166 - root_mean_squared_error: 0.0666 - val_loss: 0.0045 - val_mean_absolute_error: 0.0524 - val_r2_score: 0.7165 - val_root_mean_squared_error: 0.0673
Epoch 4/8
3174/3174 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.0039 - mean_absolute_error: 0.0482 - r2_score: 0.7537 - root_mean_squared_error: 0.0621 - val_loss: 0.0043 - val_mean_absolute_error: 0.05

In [233]:
lstm_history_df = pd.DataFrame(lstm_history.history)
lstm_history_df.to_csv('../../frontend/resources/models/history/csv/lstm.csv', index=False)
lstm_history_df

,loss,mean_absolute_error,r2_score,root_mean_squared_error,val_loss,val_mean_absolute_error,val_r2_score,val_root_mean_squared_error
0,0.005166,0.056498,0.669853,0.071876,0.005180,0.057065,0.675356,0.071970
1,0.004723,0.054611,0.698159,0.068726,0.004908,0.055393,0.692411,0.070054
2,0.004283,0.051480,0.726317,0.065443,0.004524,0.052368,0.716482,0.067257
3,0.003769,0.047488,0.759128,0.061394,0.004332,0.050708,0.728484,0.065818
4,0.003524,0.045530,0.774832,0.059361,0.004353,0.050718,0.727181,0.065976
5,0.003394,0.044501,0.783133,0.058254,0.004288,0.050338,0.731223,0.065486
6,0.003310,0.043839,0.788486,0.057531,0.004337,0.050401,0.728147,0.065859
7,0.003249,0.043351,0.792357,0.057001,0.004174,0.049313,0.738377,0.064608


In [221]:
lstm_history_df_scaled = pd.DataFrame(scaler.fit_transform(lstm_history_df), columns=lstm_history_df.columns)
lstm_history_df_scaled

,loss,mean_absolute_error,r2_score,root_mean_squared_error,val_loss,val_mean_absolute_error,val_r2_score,val_root_mean_squared_error
0,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
1,0.768967,0.856437,0.231058,0.788233,0.729378,0.784324,0.270622,0.739755
2,0.539211,0.618331,0.460912,0.567540,0.347429,0.394051,0.652570,0.359847
3,0.271271,0.314674,0.728746,0.295290,0.156982,0.179981,0.843018,0.164386
4,0.143213,0.165725,0.856939,0.158617,0.177662,0.181238,0.822338,0.185817
5,0.075321,0.087478,0.924703,0.084224,0.113512,0.132199,0.886488,0.119169
6,0.031645,0.037148,0.968395,0.035609,0.162333,0.140360,0.837666,0.169937
7,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000


In [114]:
lstm_metrics = train_model_lstm.evaluate(X_test_seq, y_test_seq)

21849/21849 ━━━━━━━━━━━━━━━━━━━━ 44s 2ms/step - loss: 0.0047 - mean_absolute_error: 0.0525 - r2_score: 0.6518 - root_mean_squared_error: 0.0688


In [115]:
lstm_y_pred_seq = train_model_lstm.predict(X_test_seq)

21849/21849 ━━━━━━━━━━━━━━━━━━━━ 20s 923us/step


In [241]:
lstm_metrics_df = helpers.metrics_df('LSTM', lstm_metrics)
lstm_metrics_df.to_csv('../../frontend/resources/models/metrics/lstm.csv', index=False)
lstm_metrics_df

,Model,MSE,MAE,RMSE,R2
0,LSTM,0.004907,0.053742,0.070051,0.694023


In [180]:
lstm_y_pred, lstm_y_test, lstm_difference = helpers.inverse_predictions(lstm_y_pred_seq, y_test_seq, scaler_y_train)

In [234]:
lstm_daily_stats = helpers.daily_stats(lstm_y_pred, lstm_y_test, lstm_difference, test_dates_seq)
lstm_daily_stats.to_csv('../../frontend/resources/models/daily_stats/csv/lstm.csv', index=False)
lstm_daily_stats.head()

,fecha,actual_mean,actual_min,actual_max,actual_std,predicted_mean,predicted_min,predicted_max,predicted_std
0,2023-01-01,12.538392,1.70042,22.19802,3.418742,11.890576,5.618233,22.268463,1.867049
1,2023-01-02,10.296912,-0.80051,21.30125,4.043459,10.213053,2.745819,19.552053,1.793805
2,2023-01-03,9.300819,-2.19858,21.80255,4.564409,9.676287,2.534132,21.073025,2.325640
3,2023-01-04,9.353709,-0.49973,21.40151,4.595705,9.549453,4.184849,20.662832,2.050872
4,2023-01-05,8.972093,-0.80051,21.40151,4.497757,9.384128,2.595671,20.441244,2.238374


In [ ]:
lstm_daily_stats_fig = helpers.daily_stats_comparison('LSTM', lstm_daily_stats)
lstm_daily_stats_fig.write_html('../../frontend/resources/models/daily_stats/figures/lstm.html')
lstm_daily_stats_fig.show()

In [ ]:
lstm_difference_fig = helpers.difference_histogram('LSTM', lstm_difference)
lstm_difference_fig.write_html('../../frontend/resources/models/differences/lstm.html')
lstm_difference_fig.show()

In [ ]:
lstm_training_history_fig = helpers.training_history('LSTM', lstm_history.history)
lstm_training_history_fig.write_html('../../frontend/resources/models/history/figures/lstm.html')
lstm_training_history_fig.show()

In [ ]:
lstm_training_history_scaled_fig = helpers.training_history_scaled('LSTM', lstm_history_df_scaled, lstm_history_df)
lstm_training_history_scaled_fig.write_html('../../frontend/resources/models/history/figures/lstm_scaled.html')
lstm_training_history_scaled_fig.show()

In [18]:
final_model_lstm = create_lstm(shape)
final_model_lstm.compile(optimizer=Nadam(learning_rate=0.01), loss='mean_squared_error', metrics=metrics)

In [19]:
K.clear_session()
tf.compat.v1.reset_default_graph()
final_model_lstm.fit(X_full_seq, y_full_seq, epochs=5, batch_size=batch_size)

Epoch 1/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0065 - mean_absolute_error: 0.0596 - r2_score: 0.5761 - root_mean_squared_error: 0.0780 
Epoch 2/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - loss: 0.0048 - mean_absolute_error: 0.0548 - r2_score: 0.6872 - root_mean_squared_error: 0.0691
Epoch 3/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - loss: 0.0044 - mean_absolute_error: 0.0520 - r2_score: 0.7130 - root_mean_squared_error: 0.0662
Epoch 4/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - loss: 0.0039 - mean_absolute_error: 0.0483 - r2_score: 0.7445 - root_mean_squared_error: 0.0624
Epoch 5/5
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - loss: 0.0037 - mean_absolute_error: 0.0465 - r2_score: 0.7591 - root_mean_squared_error: 0.0606


In [20]:
final_model_lstm.save('../../backend/data/ml/models/lstm.keras')